In [29]:
import warnings
warnings.filterwarnings(action='ignore')
import pymongo
import requests

starbucks 사이트에서 시도 목록을 얻어와서 starbucks 데이터베이스의 sido 컬렉션에 저장한다.

In [30]:
mongoConnection = pymongo.MongoClient('mongodb://localhost:27017/') # mongoDB에 연결한다.
starbucks = mongoConnection['starbucks'] # starbucks 데이터베이스에 연결한다.
sido = starbucks['sido'] # starbucks 데이터베이스 sido 컬렉션을 얻어온다.
sido.delete_many({}) # sido 컬렉션에 저장된 모든 데이터를 제거한다.

request = requests.post('https://www.starbucks.co.kr/store/getSidoList.do').json()
# print(type(request))
sidoList = []
for r in request['list']:
    d = {'sido_cd': r['sido_cd'].strip(), 'sido_nm': r['sido_nm'].strip()}
    sidoList.append(d)
# print(sidoList)

# mongoDB에 데이터를 저장한다.
for s in sidoList:
    sido.insert_one(s)

sidoList = sido.find({}, {'_id': 0, 'sido_cd': 1, 'sido_nm': 1})
for s in sidoList:
    print(s['sido_nm'], end=' ')

서울 경기 광주 대구 대전 부산 울산 인천 강원 경남 경북 전남 전북 충남 충북 제주 세종 

시도 이름을 입력받아 입력받은 시도의 구군 코드를 얻어와서 starbucks 데이터베이스의 gugun 컬렉션에 저장한다.

In [31]:
sidoName = input('구군코드를 검색하려는 시도 이름을 입력하세요(예: 서울): ')
sido_cd = sido.find({'sido_nm': sidoName}, {'_id': 0, 'sido_cd': 1, 'sido_nm': 1}).next()
# print(sido_cd['sido_cd'])

request = requests.post('https://www.starbucks.co.kr/store/getGugunList.do', data={
    'sido_cd': sido_cd['sido_cd']
}).json()
gugunList = []
for r in request['list']:
    if r['gugun_nm'] is None: # 세종 None 처리
        r['gugun_nm'] = ''
    d = {'gugun_cd': r['gugun_cd'].strip(), 'gugun_nm': r['gugun_nm'].strip()}
    gugunList.append(d)
# print(gugunList)

gugun = starbucks['gugun'] # gugun 컬렉션을 만든다.
gugun.delete_many({}) # gugun 컬렉션에 저장된 모든 데이터를 제거한다.
for g in gugunList:
    gugun.insert_one(g)
    
gugunList = gugun.find({}, {'_id': 0, 'gugun_cd': 1, 'gugun_nm': 1})
for s in gugunList:
    print('[{}]'.format(s['gugun_nm']), end=' ')

구군코드를 검색하려는 시도 이름을 입력하세요(예: 서울): 서울
[강남구] [강동구] [강북구] [강서구] [관악구] [광진구] [구로구] [금천구] [노원구] [도봉구] [동대문구] [동작구] [마포구] [서대문구] [서초구] [성동구] [성북구] [송파구] [양천구] [영등포구] [용산구] [은평구] [종로구] [중구] [중랑구] 

시도 이름과 구군 이름을 입력받아 입력받은 구군 코드를 얻어와서 매장 정보를 starbucks 데이터베이스의 store 컬렉션에 저장한다.

In [33]:
# sidoName = input('구군코드를 검색하려는 시도 이름을 입력하세요(예: 서울): ')
# sido_cd = sido.find({'sido_nm': sidoName}, {'_id': 0, 'sido_cd': 1, 'sido_nm': 1}).next()
gugunName = input('구군별 매장을 검색하려는 구군 이름을 입력하세요(예: 가평군): ')
gugun_cd = gugun.find({'gugun_nm': gugunName}, {'_id': 0, 'gugun_cd': 1, 'gugun_nm': 1}).next()
# print(gugun_cd['gugun_cd'])

request = requests.post('https://www.starbucks.co.kr/store/getStore.do?r=XBOGDH6O1Z', data={
    'ins_lat': 37.5750656,
    'ins_lng': 126.9694464,
    'p_sido_cd': '',
    'p_gugun_cd': gugun_cd['gugun_cd'],
    'in_biz_cd': '',
    'iend': 2000,
    'set_date': ''
}).json()
storeList = []
for r in request['list']:
    d = {'s_name': r['s_name'].strip(), 'doro_address': r['doro_address'].strip(), 'lat': r['lat'], 'lot': r['lot']}
    storeList.append(d)
# print(storeList)

store = starbucks['store'] # store 컬렉션을 만든다.
store.delete_many({}) # store 컬렉션에 저장된 모든 데이터를 제거한다.
for s in storeList:
    store.insert_one(s)
    
storeList = store.find({}, {'_id': 0})
for s in storeList:
    print(s)

구군별 매장을 검색하려는 구군 이름을 입력하세요(예: 가평군): 종로구
{'s_name': '서대문역', 'doro_address': '서울특별시 종로구 통일로 134 (평동)', 'lat': '37.56670', 'lot': '126.96629'}
{'s_name': '서울대치과병원', 'doro_address': '서울특별시 종로구 대학로 101 서울대학교 치과병원 신관1층', 'lat': '37.57790', 'lot': '126.99965'}
{'s_name': '동숭길입구', 'doro_address': '서울특별시 종로구 대학로 146 (혜화동)', 'lat': '37.584236', 'lot': '127.002014'}
{'s_name': '강북삼성병원1층', 'doro_address': '서울특별시 종로구 새문안로 29 (평동)', 'lat': '37.568546', 'lot': '126.96756'}
{'s_name': '종로플레이스', 'doro_address': '서울특별시 종로구 창경궁로 120 (인의동)', 'lat': '37.57319', 'lot': '126.998309'}
{'s_name': '정동', 'doro_address': '서울특별시 종로구 새문안로 42 (신문로2가)', 'lat': '37.568578', 'lot': '126.970067'}
{'s_name': '광화문교보문고', 'doro_address': '서울특별시 종로구 종로 1 (종로1가)', 'lat': '37.571017', 'lot': '126.977937'}
{'s_name': '종로관수', 'doro_address': '서울특별시 종로구 삼일대로20길 13 (관수동)', 'lat': '37.569385', 'lot': '126.988665'}
{'s_name': '경복궁사거리', 'doro_address': '서울특별시 종로구 율곡로 6, B동 101호(중학동, 트윈트리타워)', 'lat': '37.575578', 'lot': '126.980581'}


시도 이름을 입력받아 입력받은 시도 코드를 얻어와서 시도 전체 매장 정보를 starbucks 데이터베이스의 store_sido 컬렉션에 저장한다.

In [34]:
sidoName = input('시도별 매장 정보를 검색하려는 시도 이름을 입력하세요(예: 서울): ')
sido_cd = sido.find({'sido_nm': sidoName}, {'_id': 0, 'sido_cd': 1, 'sido_nm': 1}).next()

request = requests.post('https://www.starbucks.co.kr/store/getStore.do?r=XBOGDH6O1Z', data={
    'ins_lat': 37.5750656,
    'ins_lng': 126.9694464,
    'p_sido_cd': sido_cd['sido_cd'],
    'p_gugun_cd': '',
    'in_biz_cd': '',
    'iend': 2000,
    'set_date': ''
}).json()
storeList = []
for r in request['list']:
    d = {'s_name': r['s_name'].strip(), 'doro_address': r['doro_address'].strip(), 'lat': r['lat'], 'lot': r['lot']}
    storeList.append(d)
    
store_sido = starbucks['store_sido'] # store_sido 컬렉션을 만든다.
store_sido.delete_many({}) # store_sido 컬렉션에 저장된 모든 데이터를 제거한다.
for s in storeList:
    store_sido.insert_one(s)
    
storeList = store_sido.find({}, {'_id': 0})
for s in storeList:
    print(s)

구군코드를 검색하려는 시도 이름을 입력하세요(예: 서울): 서울
{'s_name': '역삼아레나빌딩', 'doro_address': '서울특별시 강남구 언주로 425 (역삼동)', 'lat': '37.501087', 'lot': '127.043069'}
{'s_name': '논현역사거리', 'doro_address': '서울특별시 강남구 강남대로 538 (논현동)', 'lat': '37.510178', 'lot': '127.022223'}
{'s_name': '신사역성일빌딩', 'doro_address': '서울특별시 강남구 강남대로 584 (논현동)', 'lat': '37.5139309', 'lot': '127.0206057'}
{'s_name': '국기원사거리', 'doro_address': '서울특별시 강남구 테헤란로 125 (역삼동)', 'lat': '37.499517', 'lot': '127.031495'}
{'s_name': '대치재경빌딩R', 'doro_address': '서울특별시 강남구 남부순환로 2947 (대치동)', 'lat': '37.494668', 'lot': '127.062583'}
{'s_name': '봉은사역', 'doro_address': '서울특별시 강남구 봉은사로 619 (삼성동)', 'lat': '37.515000', 'lot': '127.063196'}
{'s_name': '압구정윤성빌딩', 'doro_address': '서울특별시 강남구 논현로 834 (신사동)', 'lat': '37.5227934', 'lot': '127.0286009'}
{'s_name': '코엑스별마당', 'doro_address': '서울특별시 강남구 영동대로 513 (삼성동)', 'lat': '37.510150', 'lot': '127.060275'}
{'s_name': '삼성역섬유센터R', 'doro_address': '서울특별시 강남구 테헤란로 518 (대치동)', 'lat': '37.507750', 'lot': '127.060651'}
{'

전국 전체 매장 정보를 starbucks 데이터베이스의 store_all 컬렉션에 저장한다.

In [35]:
request = requests.post('https://www.starbucks.co.kr/store/getStore.do?r=XBOGDH6O1Z', data={
    'ins_lat': 37.5750656,
    'ins_lng': 126.9694464,
    'p_sido_cd': '',
    'p_gugun_cd': '',
    'in_biz_cd': '',
    'iend': 2000,
    'set_date': ''
}).json()
storeList = []
for r in request['list']:
    d = {'s_name': r['s_name'].strip(), 'doro_address': r['doro_address'].strip(), 'lat': r['lat'], 'lot': r['lot']}
    storeList.append(d)
    
store_all = starbucks['store_all'] # store_all 컬렉션을 만든다.
store_all.delete_many({}) # store_all 컬렉션에 저장된 모든 데이터를 제거한다.
for s in storeList:
    store_all.insert_one(s)
    
storeList = store_all.find({}, {'_id': 0})
for s in storeList:
    print(s)

{'s_name': '송파마천사거리DT', 'doro_address': '서울특별시 송파구 거마로 78 (마천동)', 'lat': '37.499829', 'lot': '127.147672'}
{'s_name': '그랜드조선 부산', 'doro_address': '부산광역시 해운대구 해운대해변로 292 (중동)', 'lat': '35.1600338', 'lot': '129.1630512'}
{'s_name': '서울웨이브아트센터', 'doro_address': '서울특별시 서초구 잠원로 145-35 (잠원동)', 'lat': '37.518807', 'lot': '127.00704'}
{'s_name': '명동남산', 'doro_address': '서울특별시 중구 퇴계로 132 (남산동3가)', 'lat': '37.560809', 'lot': '126.986397'}
{'s_name': '센텀몰1F', 'doro_address': '부산광역시 해운대구 센텀4로 15 (우동)', 'lat': '35.16981023', 'lot': '129.12778297'}
{'s_name': '송도컨벤시아대로DT', 'doro_address': '인천광역시 연수구 벤처로 10 (송도동)', 'lat': '37.3824512554796', 'lot': '126.636816667557'}
{'s_name': '세종충남대학교병원', 'doro_address': '세종특별자치시 보듬7로 20 (도담동) 헬스케어센터 1층', 'lat': '36.51936', 'lot': '127.25828'}
{'s_name': '동탄호수공원', 'doro_address': '경기도 화성시 동탄대로 181 (송동)', 'lat': '37.1721616', 'lot': '127.1061181'}
{'s_name': '대구228중앙공원', 'doro_address': '대구광역시 중구 동성로2길 95 (동성로2가)', 'lat': '35.8701932', 'lot': '128.5969103'}
{'s_nam